In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from path import Path
from sklearn import tree

from collections import Counter

In [20]:
# Reading the file into Dataframe
df_loan = pd.read_csv('https://finalprojectanalysis-bucket.s3.us-east-2.amazonaws.com/clean_data.csv')
df_loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [21]:
# Identifying the outliers,spread
df_loan.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,505.000000,505.000000,505.000000,505.000000,505.000000
mean,5411.427723,1562.978059,144.566337,341.869307,0.859406
std,6127.874416,2564.980770,81.306177,64.900341,0.347947
min,150.000000,0.000000,9.000000,36.000000,0.000000
25%,2895.000000,0.000000,100.000000,360.000000,1.000000
50%,3833.000000,1126.000000,128.000000,360.000000,1.000000
75%,5818.000000,2253.000000,167.000000,360.000000,1.000000
max,81000.000000,33837.000000,600.000000,480.000000,1.000000


In [22]:
# Removing outliers
df_loan = df_loan.loc[(df_loan["ApplicantIncome"] < 50000)] 

In [23]:
df_loan = df_loan.loc[(df_loan["ApplicantIncome"] > 1000)]

In [24]:
df_loan.shape

(499, 12)

In [25]:
# Creating a new column
df_loan ['Total_Income'] = df_loan ['ApplicantIncome'] + df_loan ['CoapplicantIncome']

In [26]:
df_loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income
0,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,6091.0
1,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,3000.0
2,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,4941.0
3,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,6000.0
4,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,9613.0


In [27]:
#pd.getdummies to encode
df_loan_encoded = pd.get_dummies(df_loan, columns=["Gender","Education","Dependents","Property_Area","Self_Employed","Married"])

In [28]:
# Creating Encode instancce 
lbl_encode = LabelEncoder()

In [29]:
# Encode Loan_Status Column
df_loan_encoded['Loan_Status_label']=lbl_encode.fit_transform(df_loan_encoded['Loan_Status'])

In [30]:
df_loan_encoded.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Total_Income,Gender_Female,Gender_Male,Education_Graduate,...,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Self_Employed_No,Self_Employed_Other,Self_Employed_Yes,Married_No,Married_Yes,Loan_Status_label
0,4583,1508.0,128.0,360.0,1.0,N,6091.0,0,1,1,...,0,1,0,0,1,0,0,0,1,0
1,3000,0.0,66.0,360.0,1.0,Y,3000.0,0,1,1,...,0,0,0,1,0,0,1,0,1,1
2,2583,2358.0,120.0,360.0,1.0,Y,4941.0,0,1,0,...,0,0,0,1,1,0,0,0,1,1
3,6000,0.0,141.0,360.0,1.0,Y,6000.0,0,1,1,...,0,0,0,1,1,0,0,1,0,1
4,5417,4196.0,267.0,360.0,1.0,Y,9613.0,0,1,1,...,0,0,0,1,0,0,1,0,1,1


In [31]:
# Define features
y = df_loan_encoded["Loan_Status_label"]
X = df_loan_encoded.drop(columns=['Loan_Status','Loan_Status_label'])

In [32]:
Counter(y)

Counter({0: 149, 1: 350})

In [14]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [19]:
Counter(y_train)

Counter({1: 262, 0: 112})

In [104]:
# Stadardizing and scaling
from sklearn.preprocessing import StandardScaler
# Creating stadard scaler instance
sc = StandardScaler()

# Fit and transform
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [105]:
# Insantiate model  SKLearn
classifier = LogisticRegression(solver='lbfgs',
   max_iter=400,
   random_state=1)


In [106]:
X_train.shape

(374, 22)

In [107]:
# Train the model
classifier.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=400, random_state=1)

In [108]:
y_pred = classifier.predict(X_test_scaled)

In [109]:
# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(confusion_matrix(y_test, y_pred))
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.832
-----------------------------
Confusion Matrix
                             
[[19 18]
 [ 3 85]]
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.86      0.51      0.64        37
           1       0.83      0.97      0.89        88

    accuracy                           0.83       125
   macro avg       0.84      0.74      0.77       125
weighted avg       0.84      0.83      0.82       125



In [110]:
## DECISION TREE MODEL

In [111]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [112]:
# Making predictions using the testing data.
y_pred = model.predict(X_test_scaled)

In [113]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)

0.712


In [114]:
# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(confusion_matrix(y_test, y_pred))
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.712
-----------------------------
Confusion Matrix
                             
[[22 15]
 [21 67]]
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.51      0.59      0.55        37
           1       0.82      0.76      0.79        88

    accuracy                           0.71       125
   macro avg       0.66      0.68      0.67       125
weighted avg       0.73      0.71      0.72       125



In [115]:
#random forest


In [116]:
# Create a random forest classifier.
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=1) 

In [117]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [118]:
# Making predictions using the testing data.
y_pred = rf_model.predict(X_test_scaled)

In [119]:
# Calculating the accuracy score.
acc_score=print(accuracy_score(y_test, y_pred))

0.8


In [120]:
# Displaying results
print("Confusion Matrix")

print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix
Accuracy Score : None
Classification Report
              precision    recall  f1-score   support

           0       0.71      0.54      0.62        37
           1       0.82      0.91      0.86        88

    accuracy                           0.80       125
   macro avg       0.77      0.72      0.74       125
weighted avg       0.79      0.80      0.79       125



In [121]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.19507978810114548, 'Credit_History'),
 (0.1629289004350371, 'Total_Income'),
 (0.14139631881868223, 'ApplicantIncome'),
 (0.13916755667919686, 'LoanAmount'),
 (0.08826160837046838, 'CoapplicantIncome'),
 (0.04228248535323366, 'Loan_Amount_Term'),
 (0.02489126609041312, 'Property_Area_Semiurban'),
 (0.023503332849819564, 'Property_Area_Rural'),
 (0.01725852601693233, 'Dependents_0'),
 (0.0163504786635834, 'Married_Yes'),
 (0.016138156559236404, 'Education_Not Graduate'),
 (0.01568794412807607, 'Property_Area_Urban'),
 (0.015104558757299475, 'Education_Graduate'),
 (0.015086366972265298, 'Married_No'),
 (0.01472819419253686, 'Gender_Male'),
 (0.01388358422619848, 'Self_Employed_No'),
 (0.01301672548483208, 'Dependents_1'),
 (0.011796851820962127, 'Dependents_2'),
 (0.010612421412394444, 'Self_Employed_Yes'),
 (0.00962041381820268, 'Gender_Female'),
 (0.008622491894102541, 'Dependents_3+'),
 (0.004582029355381407, 'Self_Employed_Other')]

In [122]:
# Gradient BOOST

In [123]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=1)
    classifier.fit(X_train_scaled, y_train)
    
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))
    

Learning rate:  0.05
Accuracy score (training): 0.824
Accuracy score (validation): 0.816
Learning rate:  0.1
Accuracy score (training): 0.840
Accuracy score (validation): 0.824
Learning rate:  0.25
Accuracy score (training): 0.882
Accuracy score (validation): 0.800
Learning rate:  0.5
Accuracy score (training): 0.920
Accuracy score (validation): 0.784
Learning rate:  0.75
Accuracy score (training): 0.952
Accuracy score (validation): 0.784
Learning rate:  1
Accuracy score (training): 0.971
Accuracy score (validation): 0.792


In [124]:
classifier = GradientBoostingClassifier(n_estimators=40,
   learning_rate=0.25, max_features=5, max_depth=3, random_state=1)

classifier.fit(X_train_scaled, y_train)
y_pred = classifier.predict(X_test_scaled)

In [125]:
# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(confusion_matrix(y_test, y_pred))
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.824
-----------------------------
Confusion Matrix
                             
[[22 15]
 [ 7 81]]
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.76      0.59      0.67        37
           1       0.84      0.92      0.88        88

    accuracy                           0.82       125
   macro avg       0.80      0.76      0.77       125
weighted avg       0.82      0.82      0.82       125

